In [ ]:
train_labels_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Training/labels'
val_labels_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels'

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import matplotlib.colors as mcolors

# Function to read TIFF file and convert it to a numpy array
def tiff_to_numpy(file_path):
    with Image.open(file_path) as img:
        numpy_array = np.array(img)
    return numpy_array

# Function to plot the numpy array with a colormap
def plot_segmentation(numpy_array):
    # Create a colormap with distinct colors for each unique value
    unique_values = np.unique(numpy_array)
    colormap = plt.get_cmap('tab20', len(unique_values))  # Using 'tab20' colormap

    # Create a normalization object that maps the unique values to the colormap
    norm = mcolors.BoundaryNorm(boundaries=np.arange(len(unique_values)+1)-0.5, ncolors=len(unique_values))

    # Plot the image
    plt.imshow(numpy_array, cmap=colormap, norm=norm)
    plt.colorbar(ticks=unique_values)
    plt.axis('off')  # Hide axes
    plt.show()

# Example usage
file_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels/cell_00001_label.tiff'
numpy_array = tiff_to_numpy(file_path)
plot_segmentation(numpy_array)


In [ ]:
import numpy as np
from PIL import Image
import os

# Function to read TIFF file and convert it to a numpy array
def tiff_to_numpy(file_path):
    with Image.open(file_path) as img:
        numpy_array = np.array(img)
    return numpy_array

# Directory containing TIFF files
tiff_dir = val_labels_path  # Replace with the path to your directory

# Initialize variables to keep track of the max values
max_values = []
results = []

# Loop through each file in the directory
for file_name in os.listdir(tiff_dir):
    if file_name.endswith('.tiff'):
        file_path = os.path.join(tiff_dir, file_name)
        numpy_array = tiff_to_numpy(file_path)
        
        # Find the max value in the numpy array
        max_value = np.max(numpy_array)
        max_values.append(max_value)
        
        # Prepare the result string
        result = f'Max value in {file_name}: {max_value}'
        results.append(result)
        print(result)

# Convert max_values list to a numpy array
max_values = np.array(max_values)

# Sort the max values from largest to smallest
sorted_max_values = np.sort(max_values)[::-1]

# Print the sorted max values
sorted_result = f'\nSorted max values across all TIFF files (largest to smallest): {sorted_max_values}'
results.append(sorted_result)
print(sorted_result)

# Compare the max values across all numpy arrays
overall_max_value = sorted_max_values[0]
overall_result = f'\nOverall max value across all TIFF files: {overall_max_value}'
results.append(overall_result)
print(overall_result)

# Save the results to a text file
output_file = 'max_values_report.txt'
with open(output_file, 'w') as f:
    for line in results:
        f.write(line + '\n')

print(f'\nResults saved to {output_file}')


Crop images

In [45]:
import numpy as np
from PIL import Image
import os

# Function to read TIFF file and convert it to a numpy array
def tiff_to_numpy(file_path):
    with Image.open(file_path) as img:
        numpy_array = np.array(img)
    return numpy_array

# Function to extract masks and bounding boxes from the numpy array
def extract_bboxes(numpy_array):
    unique_values = np.unique(numpy_array)
    bboxes = []
    #print(numpy_array.shape)

    for value in unique_values:
        if value == 0:  # Skip the background
            continue
        
        mask = numpy_array == value

        # Find bounding box
        positions = np.argwhere(mask)
        y_min, x_min = positions.min(axis=0)
        y_max, x_max = positions.max(axis=0)
        w = x_max - x_min + 1
        h = y_max - y_min + 1
        bbox = (x_min, y_min, w, h)
        bboxes.append(bbox)
    
    bboxes = sorted(bboxes, key=lambda bbox: bbox[0])
    return bboxes

# Ensure the folders are set correctly
images_folder = '/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/images'
labels_folder = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels'
crop_folder = '/home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop'

if not os.path.exists(crop_folder):
    os.makedirs(crop_folder)

i = 1
for img_name in os.listdir(images_folder):
    label_name = img_name.replace('.png', '_label.tiff')

    img_path = os.path.join(images_folder, img_name)
    label_path = os.path.join(labels_folder, label_name)

    # Debug: Ensure file paths are correct
    print(f"Processing image: {img_path}")
    print(f"Processing label: {label_path}")

    numpy_array = tiff_to_numpy(label_path)
    bboxes = extract_bboxes(numpy_array)

    if bboxes:
        box = bboxes[0]  # Take the first box
        print(box)
        x_min, y_min, w, h = box

        with Image.open(img_path) as img:
            cropped_img = img.crop((x_min, y_min, x_min + w, y_min + h))
            cropped_img.save(os.path.join(crop_folder, f'crop_{img_name}'))

        print(f"Cropped image saved: {os.path.join(crop_folder, f'crop_{img_name}')}")
    else:
        print(f"No bounding boxes found for {label_name}")
    
    if i % 50 == 0:
        print(i)
    i+=1


Processing image: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/images/cell_00006.png
Processing label: /home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels/cell_00006_label.tiff
(0, 246, 180, 209)
Cropped image saved: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop/crop_cell_00006.png
Processing image: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/images/cell_00077.png
Processing label: /home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels/cell_00077_label.tiff
(0, 886, 26, 21)
Cropped image saved: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop/crop_cell_00077.png
Processing image: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/images/cell_00070.png
Processing label: /home/icb/hanyi.zhang/NeurIPS22-CellSeg/Tuning/labels/cell_00070_label.tiff
(60, 2416, 54, 55)
Cropped image saved: /home/icb/hanyi.zhang/main_master_thesis/NeurIPS22-CellSeg-png/val/ref_crop/crop_cell_0